In [8]:
import os
import hydra
import lightning.pytorch as pl
import torch
#from puzzlefusion_plusplus.auto_aggl import AutoAgglomerative
from omegaconf import OmegaConf,open_dict
import omegaconf



In [9]:
cfg_auto_aggl = omegaconf.OmegaConf.load('config/auto_aggl.yaml')
cfg_denoiser_data = omegaconf.OmegaConf.load('config/denoiser/data.yaml')
cfg_denoiser_encode = omegaconf.OmegaConf.load('config/denoiser/encoder.yaml')
cfg_denoiser_global_config = omegaconf.OmegaConf.load('config/denoiser/global_config.yaml')
cfg_denoiser_model = omegaconf.OmegaConf.load('config/denoiser/model.yaml')

cfg_ae_vq_vae = omegaconf.OmegaConf.load('config/ae/vq_vae.yaml')
cfg_ae_global_config = omegaconf.OmegaConf.load('config/ae/global_config.yaml')
cfg_ae_model = omegaconf.OmegaConf.load('config/ae/model.yaml')


cfg = OmegaConf.merge(
    cfg_auto_aggl,
    {"denoiser": cfg_denoiser_data},
    {"denoiser": cfg_denoiser_encode},
    {"denoiser": cfg_denoiser_global_config},
    {"denoiser": cfg_denoiser_model},
    {"ae": cfg_ae_vq_vae},
    {"ae": cfg_ae_global_config},
    {"ae": cfg_ae_model},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/global_config.yaml')},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/model.yaml')},
)

In [10]:
data_home_dir = '/disk2/data/breaking_bad/'
data_type_name = 'everyday'

data_home_dir = '/disk2/data/shape_dataset/'
data_type_name = 'shape'

with open_dict(cfg):
    cfg.experiment_output_path = data_home_dir+'experiment_output/'
    cfg.denoiser.data.data_val_dir=f'{data_home_dir}data/pc_data/{data_type_name}/val/'
    #cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/everyday_epoch100_bs64/training/last.ckpt'
    cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/shape_epoch10/training/last.ckpt'
    cfg.inference_dir= data_home_dir+'results'
    cfg.denoiser.data.val_batch_size=1
    #cfg.verifier.ckpt_path= '/disk2/data/breaking-bad-dataset/output/verifier/everyday_epoch100_bs64/training/last.ckpt'

#print(OmegaConf.to_yaml(cfg))
#print(cfg.experiment_output_path)

In [29]:
# fix the seed
pl.seed_everything(cfg.test_seed, workers=True)

# create directories for inference outputs
inference_dir = os.path.join(cfg.experiment_output_path, "inference", cfg.inference_dir)
os.makedirs(inference_dir, exist_ok=True)

# initialize data
from jahn_src.data_loader_jhahn import build_test_dataloader


#from puzzlefusion_plusplus.denoiser.dataset.dataset import build_test_dataloader
#cfg.denoiser.data.matching_data_path= '/disk2/data/breaking-bad-dataset/data/matching_data'

test_loader = build_test_dataloader(cfg.denoiser)

Seed set to 123

100%|██████████| 34/34 [00:00<00:00, 302.88it/s]


In [30]:
import importlib
import jahn_src.auto_agg_jhahn
importlib.reload(jahn_src.myutil)
importlib.reload(jahn_src.auto_agg_jhahn)
from jahn_src.auto_agg_jhahn import AutoAgglomerative

#import importlib
#import puzzlefusion_plusplus.auto_aggl
#importlib.reload(puzzlefusion_plusplus.auto_aggl)
#from puzzlefusion_plusplus.auto_aggl import AutoAgglomerative

In [32]:

model = AutoAgglomerative(cfg)

denoiser_weights = torch.load(cfg.denoiser.ckpt_path)['state_dict']

model.denoiser.load_state_dict(
    {k.replace('denoiser.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('denoiser.')}
)

model.encoder.load_state_dict(
    {k.replace('encoder.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('encoder.')}
)

# load verifier weights    
#verifier_weights = torch.load(cfg.verifier.ckpt_path)['state_dict']
#model.verifier.load_state_dict({k.replace('verifier.', ''): v for k, v in verifier_weights.items()})


trainer = pl.Trainer(accelerator=cfg.accelerator, max_epochs=1, logger=False)
trainer.test(model=model, dataloaders=test_loader)


acc torch.Size([1]) tensor([0.0556], device='cuda:0')
acc_per_part torch.Size([1, 20]) tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False]],
       device='cuda:0')
loss_per_data torch.Size([1, 20]) tensor([[ 1.6350,  1.9793,  7.4201,  8.2897,  4.0073,  4.6137,  0.7022,  3.8346,
          0.1000,  1.4541,  2.2568,  0.3778, 16.7177,  0.0000,  0.8669,  0.9746,
          2.6593,  0.7483,  2.9935,  1.6588]], device='cuda:0')
scores torch.Size([1, 190, 1]) tensor([[[0.3606],
         [0.4661],
         [0.4740],
         [0.6008],
         [0.4014],
         [0.3109],
         [0.8094],
         [0.9431],
         [0.0119],
         [0.9150],
         [0.3103],
         [0.7876],
         [0.5375],
         [0.2941],
         [0.7952],
         [0.0212],
         [0.0482],
         [0.2591],
         [0.4319],
         [0.1551],
         [0.7081],
         [0.0260],
         [0.3835],
 

[{'eval/part_acc': 0.0597761794924736,
  'eval/rmse_t': 1.631917953491211,
  'eval/rmse_r': 81.20283508300781,
  'eval/shape_cd': 11.903953552246094}]